In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

In [3]:
def weight_var(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))
def bias(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")
def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1],padding="SAME")



In [4]:
x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, weight_var([5,5,1,32])) + bias([32]))
h_pool1 = max_pool(h_conv1)

In [5]:
h_conv2 = tf.nn.relu(conv2d(h_pool1, weight_var([5,5,32,64])) + bias([64]))
h_pool2 = max_pool(h_conv2)

In [6]:
W_fc1 = weight_var([7*7*64, 1024])
b_fc1 = bias([1024])

cnn_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(cnn_flat, W_fc1) + b_fc1)

In [7]:
keep_prob = tf.placeholder(tf.float32);
magic_drop = tf.nn.dropout(h_fc1, keep_prob)

In [8]:
W_fc2 = weight_var([1024, 10])
b_fc2 = bias([10])

y_ = tf.matmul(magic_drop, W_fc2) + b_fc2

In [9]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=y_))

In [10]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_pred = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))

accur = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.global_variables_initializer().run()

In [11]:
for _ in range(200):
    batch = mnist.train.next_batch(500)
    if _%10 == 0:
        train_accur = accur.eval(feed_dict={x:batch[0], y: batch[1], keep_prob: 1.0})
        print("Step %d, training accur: %g%%" %(_, 100*train_accur))
    train_step.run(feed_dict={x:batch[0], y: batch[1], keep_prob: 0.5})


Step 0, training accur: 11.6%
Step 10, training accur: 30%
Step 20, training accur: 58.8%
Step 30, training accur: 74.6%
Step 40, training accur: 80.6%
Step 50, training accur: 77.8%
Step 60, training accur: 83%
Step 70, training accur: 85.2%
Step 80, training accur: 89.4%
Step 90, training accur: 87.2%
Step 100, training accur: 91.8%
Step 110, training accur: 93.2%
Step 120, training accur: 91%
Step 130, training accur: 93.6%
Step 140, training accur: 91.8%
Step 150, training accur: 92.8%
Step 160, training accur: 93.2%
Step 170, training accur: 93%
Step 180, training accur: 94.2%
Step 190, training accur: 94%


In [12]:
print("test accur %g"%(100*accur.eval(feed_dict={x:mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})))

test accur 94.42
